Hace falta algo que indique con qué entorno vamos a trabajar

Importar lo que hace falta

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re

In [2]:
data_url = "../Data/properatti.csv"

In [3]:
data = pd.read_csv(data_url, encoding="utf-8")

In [4]:
#limpiamos los que NaN en el precio
data = data.dropna(axis=0, how='any', subset=['price_aprox_usd'])

In [5]:
#funcion para borrar outliers.
def borrar_outliers(data, columnas):
    """Solo recibo columnas con valores numericos. 
    Las columns van en forma de tupla"""
    cols_limpiar = columnas
    mask=np.ones(shape=(data.shape[0]), dtype=bool)

    for i in cols_limpiar:
        
        #calculamos cuartiles, y valores de corte
        Q1=data[i].quantile(0.25)
        Q3=data[i].quantile(0.75)
        RSI=Q3-Q1
        max_value=Q3+1.5*RSI
        min_value=Q1-1.5*RSI
        
        #ajusto el min value a mano... no puede ser negativo.
        min_value=10
        
        #filtramos por max y min
        mask=np.logical_and(mask, np.logical_and(data[i]>=min_value, data[i]<=max_value))
    return data[mask]

In [6]:
def regex_to_bool(col, reg) :
    u"""Returns a series with boolean mask result of apply the regular expresion to the column
    col : column where to apply regular expresion
    reg : regular expresion compiled
    """
    
    serie = col.apply(lambda x : x if x is np.NaN else reg.search(x))
    serie = serie.apply(lambda x : x is not None)
   
    return serie

In [7]:
def regex_to_ones(col, reg, fill = 0) :
    u"""Returns a series with ones or other value result of apply the regular expresion to the column
    the value of one will be when the regular expression search() method found a match
    the fill value (default to 0) will be when the regular expression serach() method did not found a match
    col : column where to apply regular expresion
    reg : regular expresion compiled
    """
    
    serie = col.apply(lambda x : x if x is np.NaN else reg.search(x))
    serie = serie.apply(lambda x : 1 if x is not None else fill)
   
    return serie

In [8]:
def regex_to_tags(col, reg, match, not_match = np.NaN) :
    u"""Returns a series with 'match' values result of apply the regular expresion to the column
    the 'match' value will be when the regular expression search() method found a match
    the 'not_match' value will be when the regular expression serach() method did not found a match
    col : column where to apply regular expresion
    reg : regular expresion compiled
    """
    
    serie = col.apply(lambda x : x if x is np.NaN else reg.search(x))
    serie = serie.apply(lambda x : match if x is not None else not_match)
   
    return serie

In [10]:
_pattern = 'cochera|garage|auto'
_express = re.compile(_pattern, flags = re.IGNORECASE)

work = regex_to_tags(data['description'], _express, 'cochera', '')
data['cochera'] = work

In [11]:
_pattern = 'piscina|pileta'
_express = re.compile(_pattern, flags = re.IGNORECASE)

work = regex_to_tags(data['description'], _express, 'pileta', '')
data['pileta'] = work

In [12]:
_pattern = 'parrilla'
_express = re.compile(_pattern, flags = re.IGNORECASE)

work = regex_to_tags(data['description'], _express, 'parrilla', '')
data['parrilla'] = work

In [13]:
# Crear una categoría concatenando las encontradas

data['amenities'] = data['cochera'] +' '+ data['pileta'] +' '+ data['parrilla']

In [14]:
data[['cochera', 'pileta', 'parrilla', 'amenities']]

,cochera,pileta,parrilla,amenities
0,,,,
1,cochera,,,cochera
2,,,,
3,,,,
4,,,,
...,...,...,...,...
121215,cochera,pileta,parrilla,cochera pileta parrilla
121216,cochera,pileta,parrilla,cochera pileta parrilla
121217,cochera,,parrilla,cochera parrilla
121218,,,,


In [15]:
data['amenities'].describe()

count     100810
unique         8
top             
freq       37676
Name: amenities, dtype: object

In [16]:
data['amenities'].value_counts()

                           37676
cochera                    21660
cochera pileta parrilla    11079
cochera  parrilla           8441
cochera pileta              7565
  parrilla                  5646
 pileta parrilla            4840
 pileta                     3903
Name: amenities, dtype: int64